<a href="https://colab.research.google.com/github/DCI-alxogm/ml2023-David-Nicolas364/blob/main/Proyecto%3Afinal/Proyecto_final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto capaz de detectar fake news

El objetivo de este código es que por medio del uso de redes neuronales sea capaz de aprender de nuetro conjunto de datos y por medio de una red neuronal se pueda detectar entre noticias verdaderas y noticias falsas

Imports

In [1]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=4eb3da49f7c3ad24a2c191921f10c32f5bc860efd9ffde3fe2f4158560dd6be3
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [18]:
import pandas as pd
import numpy as np
import re
from stop_words import get_stop_words
from nltk.tokenize import TweetTokenizer
from gensim.models import Word2Vec

Datos obtenidos de: https://kaggle.com/competitions/fake-news.
Estos datos ya se encuentran divididos en un set de entrenamiento y un set de pruba

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Cosas ML/train_set.csv/train.csv")

In [4]:
#df.set_index('id',inplace = True)

In [5]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


En este dataframe se presenta el título del artículo, el autor, el texto que contiene cada artículo además para este dataset se nos presenta si las noticias son verdaderas si en label contienen un 0 y es falsa si contiene un 1

In [6]:
df.shape

(20800, 5)

Verificamos que en nuestros datos no hayan datos vacios

In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
dat = df.dropna()

In [9]:
dat.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
dat

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


El siguiente paso es la eliminación de palabras que no nos aporten nada en el análisis del texto usando la librería stop_word

In [11]:
stop_words = get_stop_words("en")
punctuations = "'!()-[]{};:',#$%&/=?´+@^`~\*.-"
def quitar_palabras(texto):
  palabras = texto.split()
  palabras_buenas =[]
  for i in range(len(palabras)):
    if palabras[i] not in stop_words and palabras[i] not in punctuations:
      palabras_buenas.append(palabras[i])
  good_words = ' '.join(palabras_buenas)
  return good_words

In [12]:
def limpiar(cuerpo):
  #se usa re.compile para tener un tipo de patrón
  #en este se encuentran números
  patron = re.compile('\d' )
  patron2 = re.compile('\W' )
  #aquí se encuentran espacios vaios
  cuerpo = cuerpo.lower()
  cuerpo = patron.sub('', cuerpo)
  cuerpo = patron2.sub(' ', cuerpo)
  cuerpo = quitar_palabras(cuerpo)
  return cuerpo

In [13]:
dat['limpio'] = dat.text.map(lambda x: limpiar(x))

<ipython-input-13-2708f10d460e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['limpio'] = dat.text.map(lambda x: limpiar(x))


In [14]:
dat

,id,title,author,text,label,limpio
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,house dem aide didn t even see comey s letter ...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,ever get feeling life circles roundabout rathe...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,truth might get fired october tension intellig...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,videos civilians killed single us airstrike id...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,print iranian woman sentenced six years prison...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,rapper t unloaded black celebrities met donald...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,green bay packers lost washington redskins wee...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,macy s today grew union several great names am...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,nato russia hold parallel exercises balkans pr...


In [15]:
cuerpos = dat.limpio.values
cuerpos[1]

'ever get feeling life circles roundabout rather heads straight line toward intended destination hillary clinton remains big woman campus leafy liberal wellesley massachusetts everywhere else votes likely don inauguration dress remainder days way miss havisham forever wore wedding dress speaking great expectations hillary rodham overflowed years ago first addressed wellesley graduating class president college informed gathered students needed debate far ascertain spokesman kind like democratic primaries minus terms unknown even seven sisters school glad miss adams made clear speaking today us us miss rodham told classmates appointing edger bergen charlie mccarthys mortimer snerds attendance bespectacled granny glasses awarding matronly wisdom least john lennon wisdom took issue previous speaker despite becoming first win election seat u s senate since reconstruction edward brooke came criticism calling empathy goals protestors criticized tactics though clinton senior thesis saul alinsk

El siguiente paso es representar cada palabra de cada texto de forma vectorial

In [16]:
token = TweetTokenizer()
token.tokenize(cuerpos[1])

['ever',
 'get',
 'feeling',
 'life',
 'circles',
 'roundabout',
 'rather',
 'heads',
 'straight',
 'line',
 'toward',
 'intended',
 'destination',
 'hillary',
 'clinton',
 'remains',
 'big',
 'woman',
 'campus',
 'leafy',
 'liberal',
 'wellesley',
 'massachusetts',
 'everywhere',
 'else',
 'votes',
 'likely',
 'don',
 'inauguration',
 'dress',
 'remainder',
 'days',
 'way',
 'miss',
 'havisham',
 'forever',
 'wore',
 'wedding',
 'dress',
 'speaking',
 'great',
 'expectations',
 'hillary',
 'rodham',
 'overflowed',
 'years',
 'ago',
 'first',
 'addressed',
 'wellesley',
 'graduating',
 'class',
 'president',
 'college',
 'informed',
 'gathered',
 'students',
 'needed',
 'debate',
 'far',
 'ascertain',
 'spokesman',
 'kind',
 'like',
 'democratic',
 'primaries',
 'minus',
 'terms',
 'unknown',
 'even',
 'seven',
 'sisters',
 'school',
 'glad',
 'miss',
 'adams',
 'made',
 'clear',
 'speaking',
 'today',
 'us',
 'us',
 'miss',
 'rodham',
 'told',
 'classmates',
 'appointing',
 'edger',
 

In [17]:
lista_de_textos=[]
for i in cuerpos:
  lista_de_textos.append(token.tokenize(i))

In [28]:
np.shape(lista_de_textos)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(18285,)